# Model of work distribution in a flat organization

* no managers; only workers
* no support staff; all infrastructure for communication and tasks works without failure
* no dark patterns

TODO:
* use realistic distributions (e.g., power law) instead of uniform
* instead of searching randomly, search second-order contacts

In [1]:
import random
import sys
print(sys.version_info)

sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)


### global variables for simulation

#### user-defined

In [2]:
skill_set_for_people = ['A','B','C'] # uniform distribution
skill_set_for_tasks  = ['A','B','C'] # uniform distribution
max_skill_level_per_person      = 3  # uniform distribution
max_skill_level_per_task        = 3  # uniform distribution
max_number_of_tasks_per_process = 3  # uniform distribution
max_task_duration_in_ticks      = 3  # uniform distribution

number_of_people = 4
social_circle_size = 2

max_ticks_to_simulate = 100

#### not set by user

In [3]:
process_id = 0

### The two classes: person and process

In [4]:
class CreatePerson():
    # static variables
#    skill_set = ['A','B','C']
#    max_skill_level = 3
    
    def __init__(self, unique_id:int):
        """
        each person has a
        * uniquie_id
        * skill/specialization matrix
        """
        self.unique_id = unique_id # problem: this can be set by the user after initialization; see https://stackoverflow.com/a/48709694/1164295
        self.backlog_of_processes = []
        self.status = "idle"
        self.assigned_process = None 
        self.contact_list = None
        self.skill_specialization_dict = {}
        for skill in skill_set_for_people:
            self.skill_specialization_dict[skill] = random.randint(1,max_skill_level_per_person+1)
        return
        
    def set_status(self, status):
        """
        status is a string with value idle xor working xor coordinating
        
        As this class is currently written, user can set "status" to arbitrary values
        """
        try:
            assert(status=="idle" or 
                   status=="working" or 
                   status=="coordinating")
            self.status = status
            return "status set to '"+status+"'"
        except AssertionError:
            return "ERROR: failed to set status; use either 'idle' xor 'working' xor 'coordinating'"
        return "ERROR: failed"
        

    def add_person_to_contact_list(self, person_id):
        """
        https://en.wikipedia.org/wiki/Dunbar%27s_number
        """
        self.contact_list.append(person_id)
        if len(self.contact_list)>social_circle_size:
            self.contact_list.pop(0)
        return
    
    #def add_task_to_backlog(self, work_item):
    #    self.backlog_of_processes.append(work_item)
    #    return 
        
    #def get_next_task_from_backlog(self):
    #    return self.backlog_of_processes.pop(0)
    
    #def show_skill_specialization(self):
    #    return self.skill_specialization_dict
        
    #def show_task_backlog(self):
    #    return self.backlog_of_processes

In [5]:
new_person = CreatePerson(1)
new_person.set_status("asdf")

"ERROR: failed to set status; use either 'idle' xor 'working' xor 'coordinating'"

In [6]:
new_person.set_status("working")

"status set to 'working'"

In [7]:
list_of_people = []
for person_index in range(number_of_people):
    #this_person_dict = {}
    #for specialization in skill_set_for_people:
    #    this_person_dict[specialization] = random.choice(range(max_skill_level_per_person))
    
    list_of_people.append(CreatePerson(person_index))

In [8]:
for person in list_of_people:
    print(person.unique_id)

0
1
2
3


In [9]:
class CreateProcess():
    """
    processes are a sequence of tasks
    """
    # static variables
    #skill_set = ['A','B','C']
    #max_skill_level = 3
    
    
    def __init__(self, unique_id:int):
        self.unique_id = unique_id
        self.list_of_tasks = []
        # a process is a sequence of tasks
#        for task_id in range(process.max_number_of_tasks):
        for task_id in range(random.randint(1,max_number_of_tasks_per_process+1)):
            task_dict = {} #{"task id": task_id}
            # task is either completed, active, or waiting
            #task_dict['status'] = "waiting"
            # each task has a skill, specialization, duration
            duration = random.randint(1,max_task_duration_in_ticks+1)
            task_dict = {'process ID': unique_id,
                         'specialization': random.choice(skill_set_for_tasks),
                         'skill level': random.randint(1,max_skill_level_per_task+1),
                         'total duration': duration,
                         'remaining duration': duration}
            self.list_of_tasks.append(task_dict)
   
    def show_tasks(self):
        for task_dict in self.list_of_tasks:
            print(task_dict)
        return
    
    #def show_process(self):
    #    return self.list_of_tasks
    
    #def start_next_task(self):
    #    """
    #    removes task from process
    #    """
    #    return self.list_of_tasks.pop()
    
    #def what_is_next_task(self):
    #    return self.list_of_tasks[-1]

In [10]:
new_p = CreateProcess(101)
new_p.list_of_tasks

[{'process ID': 101,
  'specialization': 'B',
  'skill level': 1,
  'total duration': 2,
  'remaining duration': 2},
 {'process ID': 101,
  'specialization': 'C',
  'skill level': 1,
  'total duration': 2,
  'remaining duration': 2}]

instead of a static list of processes to draw from,   
create an infinite backlog of work

In [11]:
def new_process():
    global process_id
    process_id+=1
    return CreateProcess(process_id)

### Assessment tools

In [12]:
def show_all_people(list_of_people):
    for person in list_of_people:
        print('id=',person.unique_id, 
              '; skill matrix=',person.skill_specialization_dict)

In [13]:
show_all_people(list_of_people)

id= 0 ; skill matrix= {'A': 2, 'B': 2, 'C': 4}
id= 1 ; skill matrix= {'A': 1, 'B': 3, 'C': 3}
id= 2 ; skill matrix= {'A': 2, 'B': 3, 'C': 1}
id= 3 ; skill matrix= {'A': 3, 'B': 1, 'C': 2}


In [14]:
def current_status_of_people(list_of_people):
    """
    what is each person doing?
    """
    for person in list_of_people:
        try:
            task = person.assigned_process.list_of_tasks[0]
        except AttributeError:
            task = None
        print('person id=',person.unique_id, 
                  '; status=',person.status,
                  '; task=',task,
                  '; number of processes in backlog=',len(person.backlog_of_processes))

In [15]:
current_status_of_people(list_of_people)

person id= 0 ; status= idle ; task= None ; number of processes in backlog= 0
person id= 1 ; status= idle ; task= None ; number of processes in backlog= 0
person id= 2 ; status= idle ; task= None ; number of processes in backlog= 0
person id= 3 ; status= idle ; task= None ; number of processes in backlog= 0


In [16]:
def cumulative_task_backlog_size(list_of_people):
    backlog_count = 0
    for person in list_of_people:
        backlog_count += len(person.backlog_of_processes)
    return backlog_count

In [17]:
cumulative_task_backlog_size(list_of_people)

0

In [18]:
def all_idle(list_of_people):
    for person in list_of_people:
        if person.status != "idle":
            return False
    return True

In [19]:
all_idle(list_of_people)

True

### Initialize Simulation

In [20]:
# randomly distribute processes to people

for index in range(len(list_of_people)):
    #print('person ID',list_of_people[index].unique_id)
    try:
        list_of_people[index].backlog_of_processes.append(new_process())
    except IndexError:
        print("there are more people than processes")
        break

In [21]:
current_status_of_people(list_of_people)

person id= 0 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 1 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 2 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 3 ; status= idle ; task= None ; number of processes in backlog= 1


### Begin ticks of Simulation

In [22]:
def pick_a_random_person(person_index, contacts, list_of_people):
    """
    find someone who is not myself and is not someone I already know
    """
    attempts = 0
    try:
        len(contacts)
    except TypeError: # contacts is None
        contacts=[]
    while (attempts<100):
        another_person = random.choice(range(len(list_of_people)))
        if ((another_person not in contacts) and 
            (another_person != person_index)):
            return another_person
    print("failed to find another person who is not a contact")
    return None
    

In [23]:
tick=-1

while ((tick<max_ticks_to_simulate)):
    tick=tick+1
    print('\n===== tick',tick,'=====')
    print("   ===== status at the leading edge of this tick: =====")
    current_status_of_people(list_of_people)
    print("   ===== updates happening in this tick: =====")
    # each person looks in their backlog for work
    for person_index in range(len(list_of_people)):
        skill_dict = list_of_people[person_index].skill_specialization_dict
        print("person",person_index,'has skills',skill_dict)
        if list_of_people[person_index].status=="idle":
            assert(list_of_people[person_index].assigned_process is None)
            try:
                # new processes get appended, so the oldest task is at position 0
                list_of_people[person_index].assigned_process = list_of_people[person_index].backlog_of_processes.pop(0)
                print("   person",person_index,"got task from their backlog")
            except IndexError: # pop from empty list
                print("   person",person_index,"had no task in backlog; get new process from infinite queue")
                list_of_people[person_index].assigned_process = new_process()

            # can the person do the task, or do they need to coordinate?
            if (list_of_people[person_index].assigned_process.list_of_tasks[0]['skill level'] <= 
                skill_dict[list_of_people[person_index].assigned_process.list_of_tasks[0]['specialization']]): # person can do task
                print("   person",person_index,"was idle but can do the task!")
                list_of_people[person_index].status="working"
            else: # person doesn't have sufficient skill for task; person needs to find someone else
                print("   person",person_index,"was idle and does not have sufficient skill")
                list_of_people[person_index].status="coordinating"
        
        # at this point, regardless of status, the person has a task
        if not list_of_people[person_index].assigned_process:
            raise Exception("person",person_index,"should have a task")
            
        my_process = list_of_people[person_index].assigned_process
        print("   person",person_index,"has an assigned task",my_process.list_of_tasks[0])
            
        assert((list_of_people[person_index].status=="working") or 
               (list_of_people[person_index].status=="coordinating"))
        
        if list_of_people[person_index].status=="working":
            speedup = skill_dict[my_process.list_of_tasks[0]['specialization']]/my_process.list_of_tasks[0]['skill level']
            print("   speedup for person",person_index,"is", speedup)
            my_process.list_of_tasks[0]['remaining duration'] = my_process.list_of_tasks[0]['remaining duration'] - speedup # "doing the work"
            if (my_process.list_of_tasks[0]['remaining duration']<0): # task was completed
                print("   task completed!")
                list_of_people[person_index].assigned_process = None # it disappears from the simulation
                list_of_people[person_index].status = "idle"
            else: # update the task to reflect there being less work remaining because the person did some work
                list_of_people[person_index].assigned_process = my_process
            
        if list_of_people[person_index].status=="coordinating":
            contacts = list_of_people[person_index].contact_list
            print("   person",person_index,"contacts=",contacts)
            if contacts: # I know people!
                print("   person",person_index,"looks in contact list")
                for contact_id in contacts: # do the people I know have the skills to do this task?
                    contacts_skill_dict = list_of_people[contact_id].skill_specialization_dict
                    if (my_process.list_of_tasks[0]['skill level'] <= contacts_skill_dict[my_process.list_of_tasks[0]['specialization']]): # contact can do task
                        list_of_people[contact_id].backlog_of_processes.append(process)
                        list_of_people[person_index].assigned_process = None
                        list_of_people[person_index].status = "idle"
                        print("   person",person_index,"gave task to person",contact_id,"from contact list")
            # after looking through contacts, if the status is still "coordinating", 
            # then person didn't have a contact who could do the work
            if list_of_people[person_index].status=="coordinating": 
                another_person_id = pick_a_random_person(person_index, contacts, list_of_people)
                another_person_skill_dict = list_of_people[another_person_id].skill_specialization_dict
                if (my_process.list_of_tasks[0]['skill level'] <= another_person_skill_dict[my_process.list_of_tasks[0]['specialization']]): # another_person can do task
                    list_of_people[another_person_id].backlog_of_processes.append(my_process)
                    list_of_people[person_index].assigned_process = None
                    list_of_people[person_index].status = "idle"
                    print("   person",person_index,"gave task to person",another_person_id,"from random search")
                else:
                    print("   person",person_index,"wasn't able to pawn off the task to a random person")


===== tick 0 =====
   ===== status at the leading edge of this tick: =====
person id= 0 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 1 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 2 ; status= idle ; task= None ; number of processes in backlog= 1
person id= 3 ; status= idle ; task= None ; number of processes in backlog= 1
   ===== updates happening in this tick: =====
person 0 has skills {'A': 2, 'B': 2, 'C': 4}
   person 0 got task from their backlog
   person 0 was idle and does not have sufficient skill
   person 0 has an assigned task {'process ID': 1, 'specialization': 'A', 'skill level': 4, 'total duration': 1, 'remaining duration': 1}
   person 0 contacts= None
   person 0 wasn't able to pawn off the task to a random person
person 1 has skills {'A': 1, 'B': 3, 'C': 3}
   person 1 got task from their backlog
   person 1 was idle but can do the task!
   person 1 has an assigned task {'process ID': 2, 'specialization': 'C'